# Tutorial `sdswrapper`

### A Python package for spatial data science workflows.

To install use: 
```bash
!pip install sdswrapper
```

Pacotes python:

In [1]:
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor 

from pykrige.rk import RegressionKriging
from pykrige.ok import OrdinaryKriging

import sdswrapper as sds

Instanciando os modelos de interesse:

In [2]:
model_list = [
    ('LinearRegression', LinearRegression()),
    ('RidgeRegression', Ridge()),
    ('LassoRegression', Lasso()),
    ('ElasticNetRegression', ElasticNet()),
    ('DecisionTreeRegression', DecisionTreeRegressor()),
    ('RandomForestRegression', RandomForestRegressor()),
    ('GradientBoostingRegression', GradientBoostingRegressor()),
    ('SupportVectorRegression', SVR()),
    ('KNearestNeighborsRegression', KNeighborsRegressor())
]

In [3]:
krige_models = list()

for model_name, model in model_list:

    if model_name == 'DecisionTreeRegression':

        continue

    krige_models.append(
        (
            'KrigingRegressionRF' + model_name,
            RegressionKriging(
                regression_model    = model,
                variogram_model     = 'linear',
                verbose             = False,
                nlags               = 10
            )
        )
    )

In [4]:
ordinary_kriging = sds.OrdinaryKrigingInterface()


model_list.append(
    ('OrdinaryKriging', ordinary_kriging)
)

In [5]:
model_list.extend(krige_models)

In [6]:
model_list

[('LinearRegression', LinearRegression()),
 ('RidgeRegression', Ridge()),
 ('LassoRegression', Lasso()),
 ('ElasticNetRegression', ElasticNet()),
 ('DecisionTreeRegression', DecisionTreeRegressor()),
 ('RandomForestRegression', RandomForestRegressor()),
 ('GradientBoostingRegression', GradientBoostingRegressor()),
 ('SupportVectorRegression', SVR()),
 ('KNearestNeighborsRegression', KNeighborsRegressor()),
 ('OrdinaryKriging', OrdinaryKrigingInterface()),
 ('KrigingRegressionRFLinearRegression',
  <pykrige.rk.RegressionKriging at 0x153943c90>),
 ('KrigingRegressionRFRidgeRegression',
  <pykrige.rk.RegressionKriging at 0x15997ee10>),
 ('KrigingRegressionRFLassoRegression',
  <pykrige.rk.RegressionKriging at 0x15997f210>),
 ('KrigingRegressionRFElasticNetRegression',
  <pykrige.rk.RegressionKriging at 0x153015210>),
 ('KrigingRegressionRFRandomForestRegression',
  <pykrige.rk.RegressionKriging at 0x1599b2210>),
 ('KrigingRegressionRFGradientBoostingRegression',
  <pykrige.rk.RegressionKr

Dataset para testes:

In [7]:
sg = sds.SampleGenerator(
            y_filepath                      = sds.ABUNDANCE_FILEPATH,
            p_1_filepath                    = sds.P_1,
            p_2_filepath                    = sds.P_2,
            georreferenced_raster_filepath  = sds.SUITABILITY_FILEPATH
        )

In [8]:
df_y = sg.sample(10)

df_pseudoabsences = sg.sample(10, pseudoabsences=True)

dataset = pd.concat([df_y, df_pseudoabsences], axis=0)

In [9]:
dataset

,ID,coordenada_X,coordenada_Y,y,p_1,p_2
0,0.0,285.0,233.0,589.0,26.343166,2877.0
1,1.0,286.0,295.0,494.0,25.580000,3334.0
2,2.0,395.0,225.0,661.0,26.705667,2788.0
3,3.0,278.0,351.0,548.0,25.977501,3165.0
4,4.0,251.0,396.0,431.0,26.370333,2976.0
5,5.0,136.0,349.0,574.0,25.405333,3209.0
6,6.0,104.0,128.0,536.0,27.160334,2817.0
7,7.0,378.0,213.0,878.0,26.015833,2830.0
8,8.0,142.0,120.0,389.0,27.238167,2226.0
9,9.0,233.0,298.0,811.0,26.217667,2798.0


Wrapper:

In [10]:
sds_wrapper = sds.Wrapper(
    model_list              = model_list,
    dataset                 = dataset,
    X_column_names          = ['coordenada_X', 'coordenada_Y'],
    P_column_names          = ['p_1', 'p_2'],
    y_column_name           = 'y',
    projections_folder      = '/', # path to a folder dedicated to saving the projections
    k                       = 5,
    gridsearch_parameters   = None
)

In [11]:
output = sds_wrapper.fit()

Inspecionando resultados dos modelos:

In [12]:
df_output = pd.DataFrame(output).sort_values(by='model_metrics_mean', ascending=True)

In [13]:
df_output

,sample_size,name,model_type,model_metrics_mean,model_metrics_std,trained_model
5,20,RandomForestRegression,SK,110.469868,96.109718,"(DecisionTreeRegressor(max_features=1.0, rando..."
14,20,KrigingRegressionRFRandomForestRegression,KR,118.081138,75.226224,<pykrige.rk.RegressionKriging object at 0x1599...
17,20,KrigingRegressionRFKNearestNeighborsRegression,KR,120.264609,72.388672,<pykrige.rk.RegressionKriging object at 0x1599...
15,20,KrigingRegressionRFGradientBoostingRegression,KR,132.636631,133.795286,<pykrige.rk.RegressionKriging object at 0x1599...
6,20,GradientBoostingRegression,SK,134.092102,135.815358,([DecisionTreeRegressor(criterion='friedman_ms...
4,20,DecisionTreeRegression,SK,144.019120,151.620359,DecisionTreeRegressor()
8,20,KNearestNeighborsRegression,SK,149.041119,122.145538,KNeighborsRegressor()
16,20,KrigingRegressionRFSupportVectorRegression,KR,151.880805,127.159969,<pykrige.rk.RegressionKriging object at 0x1599...
9,20,OrdinaryKriging,KR,164.952042,114.728270,OrdinaryKrigingInterface()
0,20,LinearRegression,SK,166.902931,85.138419,LinearRegression()


Projetando para a América do Sul e computando a qualidade das projecções:

In [15]:
df_full_data = sg.get_full_data()

In [16]:
df_full_data

,coordenada_X,coordenada_Y,y,p_1,p_2
0,0,0,0.0,-3.400000e+38,-3.400000e+38
1,1,0,0.0,-3.400000e+38,-3.400000e+38
2,2,0,0.0,-3.400000e+38,-3.400000e+38
3,3,0,0.0,-3.400000e+38,-3.400000e+38
4,4,0,0.0,-3.400000e+38,-3.400000e+38
...,...,...,...,...,...
1920298,1136,1682,0.0,-3.400000e+38,-3.400000e+38
1920299,1137,1682,0.0,-3.400000e+38,-3.400000e+38
1920300,1138,1682,0.0,-3.400000e+38,-3.400000e+38
1920301,1139,1682,0.0,-3.400000e+38,-3.400000e+38


In [ ]:
model_id = 6

print('Model:', output[model_id]['name'])

prediction = sds_wrapper.predict(
    trained_model = output[model_id]['trained_model'],
    X = df_full_data[['coordenada_X', 'coordenada_Y']],
    p = df_full_data[['p_1', 'p_2']],
    shape = sg.y.shape
)

In [ ]:
sds_wrapper.score(
    np.where(np.isnan(prediction), -1, prediction),
    np.where(np.isnan(sg.y), -1, sg.y),
)

In [ ]:
prediction = np.where(np.isnan(sg.y), np.nan, prediction)

In [ ]:
fig = plt.figure(figsize=(15, 5))

# Plotando o mapa observado
ax1 = plt.subplot(1, 2, 1)
plt.imshow(sg.y, cmap='coolwarm', vmin=0, vmax=1000)
plt.colorbar()
plt.title('Observed', fontsize=16)

# Adicionando os pontos do dataset
plt.scatter(dataset['coordenada_X'], dataset['coordenada_Y'], 
            c='red', s=10, label='Dataset Points')
plt.legend()

# Plotando o mapa previsto
ax2 = plt.subplot(1, 2, 2)
plt.imshow(prediction, origin='lower', cmap='coolwarm', vmin=0, vmax=1000)
plt.colorbar()
plt.gca().invert_yaxis()
plt.title('Predicted', fontsize=16)

plt.tight_layout()

In [ ]:
plt.imshow( (prediction - sg.y), cmap='coolwarm', interpolation='nearest', vmin=-1000, vmax=1000)
plt.colorbar()
plt.title('Error in Predicted Y')

Gráfico do Wrapper:

In [ ]:
df_output['prediction']  = df_output['trained_model'].apply(lambda x: sds_wrapper.predict(
    trained_model = x,
    X = df_full_data[['coordenada_X', 'coordenada_Y']],
    p = df_full_data[['p_1', 'p_2']],
    shape = sg.y.shape
))

In [ ]:
df_output['prediction']  = df_output['prediction'].apply(lambda x: sds_wrapper.mask(x, sg.y))

In [ ]:
sds_wrapper.plot(df_output, reference_data = sg.y, best = True)

In [ ]:
# End #